<a href="https://colab.research.google.com/github/Aneeta-Xavier/pilates_project-/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -q \
  pytube \
  youtube-transcript-api \
  PyMuPDF \
  unstructured \
  langchain \
  langchain-community \
  langchain-openai \
  faiss-cpu \
  ragas \
  datasets \
  tavily-python \
  langchain-huggingface \
  evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import getpass

import os
from tavily import TavilyClient


from tavily import TavilyClient
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter Tavily API key: ")
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])


Enter Tavily API key: ··········


# **Loading the Data**

In this, we categorize the YT links to Full Body Reformer videos, Core, and then reformer inspired videos. Had more videos, but couldn't parse with YT.

In [ ]:
reformer_video_links = {
    "reformer_full_body": [
        "https://www.youtube.com/watch?v=3kluurmB2Kwb",
        "https://www.youtube.com/watch?v=42GsslxsMBQ",
        "https://www.youtube.com/watch?v=XMHUw3EhBlI",
        "https://www.youtube.com/watch?v=zKmrKPoKkgw"
    ],
    "reformer_core": [
        "https://www.youtube.com/watch?v=QUNM0x9Aeyg",
        "https://www.youtube.com/watch?v=cKVRVrQR-14",
        "https://www.youtube.com/watch?v=hv31gHbZtRk",
    ],
    "reformer_inspired_full_body": [
        "https://www.youtube.com/watch?v=hMN3Kvykei4",
    ],
    "reformer_inspired_core": [  # renamed from reformer_inspired_sliders
        "https://www.youtube.com/watch?v=hMN3Kvykei4&t=1s",
    ]
}



In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.documents import Document
import re

def get_video_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11})", url)
    return match.group(1) if match else None

def load_categorized_youtube_captions(video_dict):
    docs = []
    for category, urls in video_dict.items():
        for url in urls:
            video_id = get_video_id(url)
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id)
                text = "\n".join([entry['text'] for entry in transcript])
                docs.append(Document(
                    page_content=text,
                    metadata={"source": url, "category": category}
                ))
            except Exception as e:
                print(f"Failed to fetch captions for {url}: {e}")
    return docs

video_docs = load_categorized_youtube_captions(reformer_video_links)
print(f"Loaded {len(video_docs)} YouTube transcripts.")


Loaded 9 YouTube transcripts.


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 611005089-pilates-reformer-1.pdf to 611005089-pilates-reformer-1.pdf
Saving 686451944-NETA-Intro-to-Teaching-Reformer-Pilates-2008.pdf to 686451944-NETA-Intro-to-Teaching-Reformer-Pilates-2008.pdf


In [ ]:
import fitz  # PyMuPDF
from langchain_core.documents import Document

def load_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text() for page in doc])
    return Document(page_content=text, metadata={"source": file_path})

pdf_paths = list(uploaded.keys())
pdf_docs = [load_pdf(path) for path in pdf_paths]


In [ ]:
# all_docs = video_docs + pdf_docs
# print(f"✅ Total documents: {len(all_docs)} (videos + PDFs)")

In [ ]:
import json

all_docs = video_docs + pdf_docs
print(f"✅ Total documents: {len(all_docs)} (videos + PDFs)")

json_data = [{"content": doc.page_content, "metadata": doc.metadata} for doc in all_docs]

with open("combined_data.json", "w") as f:
    json.dump(json_data, f)

print("✅ Saved all documents to combined_data.json")


✅ Total documents: 11 (videos + PDFs)
✅ Saved all documents to combined_data.json


In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


In [ ]:
from google.colab import userdata
userdata.get('')

SecretNotFoundError: Secret secretName does not exist.

#Baseline Model TASKS 1-4

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
chunked_docs = splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Example query
question = "What are key reformer cues for core stability?"
response = rag_chain.invoke(question)
print(response)


{'query': 'What are key reformer cues for core stability?', 'result': 'Key reformer cues for core stability focus on alignment, breath, muscle engagement, and controlled movement. Based on the provided context, here are some essential cues:\n\n1. **Alignment:**\n   - Keep the head, neck, and shoulders tension-free.\n   - Maintain a neutral pelvis or imprint position.\n   - Ensure the ribcage is contracted and in contact with the carriage.\n\n2. **Breath:**\n   - Coordinate breath with movement, for example: “Inhale as you push back, exhale as you return.”\n   - Avoid holding the breath during exercises.\n\n3. **Muscle Engagement:**\n   - Engage the abdominals and pelvic floor throughout the movement.\n   - Activate core muscles such as the transverse abdominis, multifidi, and diaphragm.\n   - Stabilize with the outer unit muscles including the anterior and posterior oblique slings, deep longitudinal system, and lateral system.\n\n4. **Movement Control:**\n   - Move with control, avoidi

In [ ]:

768 dimensions

the
[1,
 3,
 4,
 2.9
 -20.3
 2]


In [ ]:
def run_tavily_search(query, max_results=3):
    response = tavily.search(query=query, search_depth="basic", max_results=max_results)
    return "\n\n".join([item["content"] for item in response["results"]])


question = "Explain rip to hip connection's relevance to pelvic alignment in reformer Pilates."
response = rag_chain.run(question)

if not response or "I don't know" in response or len(response.strip()) < 30:
    print("⚠️ No good answer from RAG. Falling back to Tavily search.")
    web_fallback = run_tavily_search(question)
    response = f"Here’s what I found from the web:\n\n{web_fallback}"

print(response)


<ipython-input-12-a5416fb74809>:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(question)


The "rip to hip connection" isn't explicitly mentioned in the provided context, but based on Pilates principles and the information about pelvic alignment on the reformer, I can explain the general relevance of connections between the rib cage and hips to pelvic alignment.

In reformer Pilates, maintaining proper pelvic alignment is crucial for effective and safe exercise performance. The pelvis can be in a neutral position—where the hip bones and pubic bone are in the same plane—or in an imprinted position, where the pelvis tilts posteriorly, and the pubic bone is higher than the hip bones.

The connection between the ribs and hips refers to the relationship and coordination between the rib cage and the pelvis. This connection is important because:

1. **Stabilization and Alignment:** Proper engagement of the core muscles, especially the transverse abdominis, helps stabilize the pelvis and maintain neutral alignment. The rib cage and pelvis must move in coordination to avoid compensat

In [ ]:
from google.colab import files
files.download("combined_data.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

# Combine documents
all_docs = video_docs + pdf_docs

# Convert documents to JSON-serializable format
json_data = [
    {
        "content": doc.page_content,
        "metadata": doc.metadata
    }
    for doc in all_docs
]

# Save to file
with open("combined_data.json", "w") as f:
    json.dump(json_data, f, indent=2)

print("✅ Saved all documents to combined_data.json")


✅ Saved all documents to combined_data.json


In [ ]:
from google.colab import files
files.download("combined_data.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !pip install ragas datasets evaluate


In [ ]:
from ragas.testset import TestsetGenerator
from ragas.evaluation import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Wrap your models
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())


generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(all_docs, testset_size=10)



Applying HeadlinesExtractor:   0%|          | 0/11 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/11 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/11 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/113 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/225 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
# ---- Step 2: Run Each Question Through Your RAG Chain ----
for row in dataset:
    question = row.eval_sample.user_input

    # Run your RAG pipeline
    output = rag_chain.invoke({"query": question})["result"]  # extract string
    retrieved_docs = retriever.invoke(question)

    # Store the response and retrieved contexts
    row.eval_sample.response = output
    row.eval_sample.retrieved_contexts = [doc.page_content for doc in retrieved_docs]



In [ ]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())


In [ ]:
from ragas import evaluate, RunConfig
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness,
    FactualCorrectness,
    ResponseRelevancy,
    ContextEntityRecall,
    NoiseSensitivity,
)

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
run_config = RunConfig(timeout=360)

results = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        LLMContextRecall(),
        Faithfulness(),
        FactualCorrectness(),
        ResponseRelevancy(),
        ContextEntityRecall(),
        NoiseSensitivity()
    ],
    llm=evaluator_llm,
    run_config=run_config
)

# View as DataFrame
results_df = results.to_pandas()
results_df.to_csv("ragas_eval_results.csv", index=False)
print(results_df)


Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[47]: TimeoutError()


                                           user_input  \
0   How can I safely use reformers to perform bice...   
1   how i do right leg exercise good way for pelvi...   
2   why we use two heavy Springs in Pilates reform...   
3   What is the correct positioning of the feet on...   
4   How are dumbbells incorporated into Reformer P...   
5   How does the reformer on the mat workout incor...   
6   How does the reformer on the mat workout incor...   
7   How do the VMO series exercises use the Pilate...   
8   How does the control and stabilization of the ...   
9   How does maintaining pelvic stability through ...   
10  What are the key components and safety conside...   
11  can you explain how the cat stretch is done in...   

                                   retrieved_contexts  \
0   [10 \nReformer Setup & Safety \nThere are seve...   
1   [Exercise Sequence 1:  Begin Pilates breathing...   
2   [Reformer versus Matwork  \nThere are more var...   
3   [at a diagonal over the fo

In [ ]:
results_df = results.to_pandas()

# Select only numeric columns (metrics), compute mean, round to 4 decimal places
summary = results_df.mean(numeric_only=True).round(4)

# Format as Markdown
markdown_table = "| Metric | Value |\n| --- | --- |\n"
for metric, value in summary.items():
    metric_name = metric.replace("_", " ").title()
    markdown_table += f"| {metric_name} | {value:.4f} |\n"

print("\n📊 Mean Metric Table for Certification:\n")
print(markdown_table)



📊 Mean Metric Table for Certification:

| Metric | Value |
| --- | --- |
| Context Recall | 0.3679 |
| Faithfulness | 0.5939 |
| Factual Correctness(Mode=F1) | 0.4492 |
| Answer Relevancy | 0.7967 |
| Context Entity Recall | 0.1596 |
| Noise Sensitivity(Mode=Relevant) | 0.2123 |




# Task 6



In [ ]:

# STEP 1: Generate synthetic training data using your reformer corpus
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4
import json
import tqdm

from torch.utils.data import DataLoader


# Split your full documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=20)
training_documents = splitter.split_documents(all_docs)

# Assign unique IDs to each chunk
doc_id_set = set()
for doc in training_documents:
    doc_id = str(uuid4())
    while doc_id in doc_id_set:
        doc_id = str(uuid4())
    doc.metadata["id"] = doc_id
    doc_id_set.add(doc_id)

# Prepare question generation LLM
qa_chat_model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

qa_prompt = ChatPromptTemplate.from_template("""
Given the following context, generate {n_questions} high-quality, relevant questions that could be answered using ONLY this content.
Format:
1. QUESTION ONE
2. QUESTION TWO
...

Context:
{context}
""")
question_generation_chain = qa_prompt | qa_chat_model

# Function to generate synthetic (question, doc_id) pairs
def create_questions(documents, n_questions=2):
    questions = {}
    relevant_contexts = {}
    for doc in tqdm.tqdm(documents):
        doc_id = doc.metadata["id"]
        prompt_input = {"context": doc.page_content, "n_questions": n_questions}
        response = question_generation_chain.invoke(prompt_input).content
        q_list = [q.strip()[3:] for q in response.split("\n") if q.strip().startswith(tuple("123456789"))]
        for q in q_list:
            q_id = str(uuid4())
            questions[q_id] = q
            relevant_contexts[q_id] = [doc_id]
    return questions, relevant_contexts

# Generate training data
training_questions, training_relevant_contexts = create_questions(training_documents[:60])
val_questions, val_relevant_contexts = create_questions(training_documents[60:75])
test_questions, test_relevant_contexts = create_questions(training_documents[75:])

# Assemble datasets
train_corpus = {doc.metadata["id"]: doc.page_content for doc in training_documents[:60]}
val_corpus = {doc.metadata["id"]: doc.page_content for doc in training_documents[60:75]}
test_corpus = {doc.metadata["id"]: doc.page_content for doc in training_documents[75:]}

train_dataset = {"questions": training_questions, "relevant_contexts": training_relevant_contexts, "corpus": train_corpus}
val_dataset = {"questions": val_questions, "relevant_contexts": val_relevant_contexts, "corpus": val_corpus}
test_dataset = {"questions": test_questions, "relevant_contexts": test_relevant_contexts, "corpus": test_corpus}







100%|██████████| 529/529 [09:07<00:00,  1.04s/it]


In [ ]:
with open("training_dataset.json", "w") as f: json.dump(train_dataset, f)
with open("val_dataset.json", "w") as f: json.dump(val_dataset, f)
with open("test_dataset.json", "w") as f: json.dump(test_dataset, f)

In [ ]:
import os
import getpass

# Prompt you to input the API key without exposing it
os.environ["WANDB_API_KEY"] = getpass.getpass("🔑 Enter your WandB API key: ")


🔑 Enter your WandB API key: ··········


In [ ]:
import wandb
wandb.login()


wandb: Currently logged in as: aneeta-e-xavier (aneeta-e-xavier-ex) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# STEP 2: Fine-tune Snowflake Arctic Embedding Model
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import wandb

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

# Prepare training pairs
examples = []
for q_id, q_text in training_questions.items():
    doc_id = training_relevant_contexts[q_id][0]
    doc_text = train_corpus[doc_id]
    examples.append(InputExample(texts=[q_text, doc_text]))

loader = DataLoader(examples, batch_size=10)


# Set up loss
matryoshka_dims = [768, 512, 256, 128, 64]
inner_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(model, inner_loss, matryoshka_dims=matryoshka_dims)

# Set up evaluator
evaluator = InformationRetrievalEvaluator(val_questions, val_corpus, val_relevant_contexts)


In [ ]:
# Train
EPOCHS = 30
warmup_steps = int(len(loader) * EPOCHS * 0.1)
wandb.init(project="pilates", name="finetune-task6")
model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path="finetuned_arctic_ft",
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

eval/cosine_accuracy@1,█▁
eval/cosine_accuracy@10,▁▁
eval/cosine_accuracy@3,█▁
eval/cosine_accuracy@5,▁▁
eval/cosine_map@100,█▁
eval/cosine_mrr@10,█▁
eval/cosine_ndcg@10,█▁
eval/cosine_precision@1,█▁
eval/cosine_precision@10,▁▁
eval/cosine_precision@3,█▁
eval/cosine_precision@5,▁▁


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
12,No log,No log,0.633333,0.933333,1.000000,1.000000,0.633333,0.311111,0.200000,0.100000,0.633333,0.933333,1.000000,1.000000,0.845499,0.792778,0.792778
24,No log,No log,0.766667,0.966667,1.000000,1.000000,0.766667,0.322222,0.200000,0.100000,0.766667,0.966667,1.000000,1.000000,0.897019,0.862222,0.862222
36,No log,No log,0.800000,0.966667,1.000000,1.000000,0.800000,0.322222,0.200000,0.100000,0.800000,0.966667,1.000000,1.000000,0.906418,0.875000,0.875000
48,No log,No log,0.833333,0.933333,1.000000,1.000000,0.833333,0.311111,0.200000,0.100000,0.833333,0.933333,1.000000,1.000000,0.923677,0.898333,0.898333
50,No log,No log,0.866667,0.933333,1.000000,1.000000,0.866667,0.311111,0.200000,0.100000,0.866667,0.933333,1.000000,1.000000,0.935980,0.915000,0.915000
60,No log,No log,0.700000,0.933333,0.966667,1.000000,0.700000,0.311111,0.193333,0.100000,0.700000,0.933333,0.966667,1.000000,0.863257,0.817778,0.817778
72,No log,No log,0.800000,0.933333,0.966667,1.000000,0.800000,0.311111,0.193333,0.100000,0.800000,0.933333,0.966667,1.000000,0.901625,0.869444,0.869444
84,No log,No log,0.766667,0.866667,1.000000,1.000000,0.766667,0.288889,0.200000,0.100000,0.766667,0.866667,1.000000,1.000000,0.881358,0.842778,0.842778
96,No log,No log,0.733333,0.866667,1.000000,1.000000,0.733333,0.288889,0.200000,0.100000,0.733333,0.866667,1.000000,1.000000,0.867595,0.824444,0.824444
100,No log,No log,0.700000,0.933333,1.000000,1.000000,0.700000,0.311111,0.200000,0.100000,0.700000,0.933333,1.000000,1.000000,0.859914,0.813333,0.813333


In [ ]:
# STEP 3: Push Model to Hugging Face
from huggingface_hub import notebook_login
notebook_login()  # ← do this in Colab or notebook interface

In [ ]:
# Define the repo_id of your already-created model
repo_id = "AneetaXavier/reformer-pilates-embed-ft-49fc1835-9968-433d-9c45-1538ea91dcc9"

# Push to existing repo
model.push_to_hub(repo_id, exist_ok=True)


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/AneetaXavier/reformer-pilates-embed-ft-49fc1835-9968-433d-9c45-1538ea91dcc9/commit/34b7b1f0427c05109185580a47ecff56552a4eaa'

In [ ]:
!pip install langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    ContextPrecision,
    LLMContextRecall,
    ContextEntityRecall,
    NoiseSensitivity,
)
from ragas.evaluation import evaluate
from datasets import Dataset
import pandas as pd

# Load fine-tuned embeddings
finetuned_embed = HuggingFaceEmbeddings(
    model_name="AneetaXavier/reformer-pilates-embed-ft-49fc1835-9968-433d-9c45-1538ea91dcc9"
)
vectorstore_ft = FAISS.from_documents(all_docs, finetuned_embed)
retriever_ft = vectorstore_ft.as_retriever(search_kwargs={"k": 3})  # limit retrieved docs to 3
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Use RetrievalQA with LangChain's invoke method
rag_chain_ft = RetrievalQA.from_chain_type(llm=llm, retriever=retriever_ft)

# Store evaluation records
records_ft = []

for ex in dataset:
    try:
        question = ex.eval_sample.user_input
        ground_truth = getattr(ex.eval_sample, "ground_truth", "")
        difficulty = getattr(ex.eval_sample, "difficulty", "N/A")

        # Run RAG chain and limit input
        answer = rag_chain_ft.invoke({"query": question})["result"]

        # Truncate context to avoid token overload
        retrieved_docs = retriever_ft.invoke(question)
        contexts = [doc.page_content[:1000] for doc in retrieved_docs]  # limit each doc to 1000 chars

        records_ft.append({
            "difficulty": difficulty,
            "user_input": question,
            "answer": answer,
            "contexts": contexts,
            "ground_truth": ground_truth
        })

    except Exception as e:
        print(f"❌ Skipped question: {question}\nError: {e}")

# Evaluate with RAGAS
if records_ft:
    ft_dataset = Dataset.from_pandas(pd.DataFrame(records_ft))
    ft_results = evaluate(
        dataset=ft_dataset,
        metrics=[
            Faithfulness,
            ResponseRelevancy,
            ContextPrecision,
            LLMContextRecall,
            ContextEntityRecall,
            NoiseSensitivity,
        ],
        llm=llm,
    )

    results_df = ft_results.to_pandas()
    results_df.to_csv("ragas_finetune_eval.csv", index=False)
    print("✅ Evaluation complete.\n", results_df)
else:
    print("⚠️ No successful RAG runs — all questions may have failed or been skipped.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/34.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

❌ Skipped question: how you use reformers for bicep curls and leg presses in pilates like step by step?
Error: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-S1F685NPhuxqyI86xNoGF2MG on tokens per min (TPM): Limit 30000, Requested 63777. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
❌ Skipped question: How does pelvic stability contribute to effective single leg exercises on the Reformer?
Error: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-S1F685NPhuxqyI86xNoGF2MG on tokens per min (TPM): Limit 30000, Requested 69100. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
❌ Skipped questi

AttributeError: 'property' object has no attribute 'get'

In [ ]:
import os

# Create and move into the project folder
project_name = "pilates_project"
os.makedirs(project_name, exist_ok=True)
%cd {project_name}

/content/pilates_project


In [ ]:
!git clone https://github.com/Aneeta-Xavier/pilates_project-.git
%cd pilates_project-


Cloning into 'pilates_project-'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 43 (delta 16), reused 33 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 225.86 KiB | 2.90 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/pilates_project/pilates_project-


#Task 7

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    ContextPrecision,
    LLMContextRecall,
    ContextEntityRecall,
    NoiseSensitivity
)
from ragas.evaluation import evaluate
from datasets import Dataset
import pandas as pd

# STEP 1: Load Fine-Tuned Embedding Model
finetuned_embed = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")
vectorstore_ft = FAISS.from_documents(all_docs, finetuned_embed)
retriever_ft = vectorstore_ft.as_retriever(search_kwargs={"k": 5})
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

rag_chain_ft = RetrievalQA.from_chain_type(llm=llm, retriever=retriever_ft)

# STEP 2: Run existing dataset through fine-tuned pipeline
records_ft = []
for ex in dataset:
    q = ex.eval_sample.user_input
    gt = getattr(ex.eval_sample, "ground_truth", "N/A")
    difficulty = getattr(ex.eval_sample, "difficulty", "N/A")
    ans = rag_chain_ft.run(q)
    ctxs = [d.page_content for d in retriever_ft.invoke(q)]
    records_ft.append({
        "user_input": q,
        "ground_truth": gt,
        "contexts": ctxs,
        "answer": ans,
        "difficulty": difficulty
    })

# STEP 3: Evaluate with RAGAS
ft_dataset = Dataset.from_pandas(pd.DataFrame(records_ft))
ft_results = evaluate(
    ft_dataset,
    metrics=[
        LLMContextRecall(),
        Faithfulness(),
        ResponseRelevancy(),
        ContextPrecision(),
        ContextEntityRecall(),
        NoiseSensitivity()
    ]
)

ft_results_df = ft_results.to_pandas()
ft_results_df.to_csv("ragas_post_finetune.csv", index=False)
print(ft_results_df)


Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-27-96f74cac4883>:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ans = rag_chain_ft.run(q)


Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[3]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[15]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[1]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[4]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[5]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[6]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[7]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[9]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[10]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[11]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[13]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[16]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[17]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[19]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[23]: TimeoutError()
ERROR:ragas.executor:Exception r

                                           user_input  \
0   How can I safely use reformers to perform bice...   
1   how i do right leg exercise good way for pelvi...   
2   why we use two heavy Springs in Pilates reform...   
3   What is the correct positioning of the feet on...   
4   How are dumbbells incorporated into Reformer P...   
5   How does the reformer on the mat workout incor...   
6   How does the reformer on the mat workout incor...   
7   How do the VMO series exercises use the Pilate...   
8   How does the control and stabilization of the ...   
9   How does maintaining pelvic stability through ...   
10  What are the key components and safety conside...   
11  can you explain how the cat stretch is done in...   

                                   retrieved_contexts  \
0   [[Music]\nhey guys welcome back I'm Dez and to...   
1   [hi guys thanks for joining me today we\nhave ...   
2   [[Music]\nforeign\n[Music]\nhey guys welcome b...   
3   [[Music]\nhey guys my name

In [ ]:
from ragas.run_config import RunConfig

In [ ]:
rate_friendly_config = RunConfig(
    timeout=300,          # 5 minutes max for operations
    max_retries=15,       # More retries for rate limits
    max_wait=90,          # Longer wait between retries
    max_workers=8,        # Fewer concurrent API calls
    log_tenacity=True     # Log retry attempts
)

In [ ]:
# ✅ Task 6.5: Post-Finetune Evaluation with Full RAGAS Metrics

from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from ragas.evaluation import evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    ContextPrecision,
    LLMContextRecall,
    ContextEntityRecall,
    NoiseSensitivity
)
from datasets import Dataset
import pandas as pd
from tqdm import tqdm

# STEP 1: Load Fine-Tuned Embedding Model
finetuned_embed = HuggingFaceEmbeddings(
    model_name="AneetaXavier/reformer-pilates-embed-ft-49fc1835-9968-433d-9c45-1538ea91dcc9"
)
vectorstore_ft = FAISS.from_documents(all_docs, finetuned_embed)

# Reduce k to limit token overflow during RAG
retriever_ft = vectorstore_ft.as_retriever(search_kwargs={"k": 2})
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

rag_chain_ft = RetrievalQA.from_chain_type(llm=llm, retriever=retriever_ft)

# STEP 2: Build Evaluation Records
records_ft = []

for ex in tqdm(dataset, desc="Evaluating"):
    try:
        question = ex.eval_sample.user_input
        answer = rag_chain_ft.invoke({"query": question})["result"]
        retrieved_docs = retriever_ft.invoke(question)
        contexts = [doc.page_content[:1000] for doc in retrieved_docs]  # Truncate if needed

        records_ft.append({
            "user_input": question,
            "answer": answer,
            "contexts": contexts,
        })

    except Exception as e:
        print(f"❌ Skipped question due to error: {question}\n{e}")

# STEP 3: Evaluate
if not records_ft:
    raise ValueError("❌ No records were evaluated. All responses failed or were skipped.")

ft_dataset = Dataset.from_pandas(pd.DataFrame(records_ft))

ft_results = evaluate(
    dataset=ft_dataset,
    metrics=[
        Faithfulness,
        ResponseRelevancy,
        ContextPrecision,
        LLMContextRecall(),
        ContextEntityRecall(),
        NoiseSensitivity()
    ],
    llm=llm,
    run_config=rate_friendly_config
)



# Save + Print
ft_results_df = ft_results.to_pandas()
ft_results_df.to_csv("ragas_post_finetune.csv", index=False)
print("\n✅ Post-finetune RAGAS evaluation complete.")
print(ft_results_df)

# STEP 4: Vibe Check
print("\n🧪 Vibe Check Examples")
for q in [
    "Why is a neutral spine important during footwork?",
    "What does Z-sit positioning train in Pilates?",
    "How do different footbar heights affect the kinetic chain?"
]:
    print(f"\nQ: {q}")
    print("A:", rag_chain_ft.invoke({"query": q})["result"])


Some weights of BertModel were not initialized from the model checkpoint at AneetaXavier/reformer-pilates-embed-ft-49fc1835-9968-433d-9c45-1538ea91dcc9 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating: 100%|██████████| 12/12 [01:44<00:00,  8.71s/it]


AttributeError: 'property' object has no attribute 'get'

In [ ]:
import time

records_ft = []

# Config
context_limit_chars = 700  # reasonable truncation per doc
max_docs = 3               # limit retrieved docs
sleep_seconds = 2          # throttle to avoid 429s

for i, ex in enumerate(dataset):
    try:
        question = ex.eval_sample.user_input
        ground_truth = getattr(ex.eval_sample, "ground_truth", "")
        difficulty = getattr(ex.eval_sample, "difficulty", "N/A")

        # 🔁 RAG chain: answer + context
        response = rag_chain_ft.invoke({"query": question})
        answer = response["result"]

        # 📄 Retrieved documents: trimmed
        retrieved_docs = retriever_ft.invoke(question)
        contexts = [
            doc.page_content[:context_limit_chars]
            for doc in retrieved_docs[:max_docs]
        ]

        # ✅ Store result
        records_ft.append({
            "difficulty": difficulty,
            "user_input": question,
            "answer": answer,
            "contexts": contexts,
            "ground_truth": ground_truth
        })

        print(f"✅ Done ({i+1}/{len(dataset)}): {question[:60]}...")
        time.sleep(sleep_seconds)  # ⏳ rate limit control

    except Exception as e:
        print(f"❌ Skipped ({i+1}/{len(dataset)}): {question[:60]}...\nError: {e}\n")
        time.sleep(sleep_seconds)


❌ Skipped (1/12): How can I safely use reformers to perform bicep curls and tr...
Error: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-turbo-preview in organization org-S1F685NPhuxqyI86xNoGF2MG on tokens per min (TPM): Limit 30000, Requested 63787. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

❌ Skipped (2/12): how i do right leg exercise good way for pelvic stability an...
Error: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-turbo-preview in organization org-S1F685NPhuxqyI86xNoGF2MG on tokens per min (TPM): Limit 30000, Requested 63415. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

✅ Done (3/12): why we use two heavy 

In [ ]:

# STEP 4: Save & Print Results
ft_results_df = ft_results.to_pandas()
ft_results_df.to_csv("ragas_post_finetune.csv", index=False)

print("\n✅ Post-finetune RAGAS evaluation complete.\n")
print(ft_results_df)

# STEP 5: Vibe Check – Optional Freeform Prompting
print("\n🧪 Vibe Check Examples")
print("Q1:", rag_chain_ft.invoke({"query": "Why is a neutral spine important during footwork?"})["result"])
print("Q2:", rag_chain_ft.invoke({"query": "What does Z-sit positioning train in Pilates?"})["result"])
print("Q3:", rag_chain_ft.invoke({"query": "How do different footbar heights affect the kinetic chain?"})["result"])



✅ Post-finetune RAGAS evaluation complete.

                                           user_input  \
0   How can I safely use reformers to perform bice...   
1   how i do right leg exercise good way for pelvi...   
2   why we use two heavy Springs in Pilates reform...   
3   What is the correct positioning of the feet on...   
4   How are dumbbells incorporated into Reformer P...   
5   How does the reformer on the mat workout incor...   
6   How does the reformer on the mat workout incor...   
7   How do the VMO series exercises use the Pilate...   
8   How does the control and stabilization of the ...   
9   How does maintaining pelvic stability through ...   
10  What are the key components and safety conside...   
11  can you explain how the cat stretch is done in...   

                                   retrieved_contexts  \
0   [[Music]\nhey guys welcome back I'm Dez and to...   
1   [hi guys thanks for joining me today we\nhave ...   
2   [[Music]\nforeign\n[Music]\nhey guys w

RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4-turbo-preview in organization org-S1F685NPhuxqyI86xNoGF2MG on tokens per min (TPM): Limit 30000, Requested 61099. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [1]:
# Select only numeric columns (metrics), compute mean, round to 4 decimal places
summary = ft_results_df.mean(numeric_only=True).round(4)

# Format as Markdown
markdown_table = "| Metric | Value |\n| --- | --- |\n"
for metric, value in summary.items():
    metric_name = metric.replace("_", " ").title()
    markdown_table += f"| {metric_name} | {value:.4f} |\n"

print("\n📊 Mean Metric Table for Certification:\n")
print(markdown_table)

NameError: name 'ft_results_df' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [ ]:
!git add .

error: open("drive/MyDrive/ Articles .gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/ Articles .gdoc'
fatal: adding files failed


In [ ]:
%%writefile .gitignore
    # Ignore Google Drive specific files
    *.gdoc
    *.gsheet
    *.gslides

Writing .gitignore


In [ ]:
!git add .
!git commit -m "Add project files, ignoring gdoc"

error: open("drive/MyDrive/ Articles .gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/ Articles .gdoc'
fatal: adding files failed
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@b9942645a441.(none)')


In [ ]:
!mkdir my_new_folder

In [ ]:
%cd my_new_folder

[Errno 2] No such file or directory: 'my_new_folder'
/content/my_new_folder


In [ ]:
%cd /content/my_new_folder

/content/my_new_folder


In [ ]:
# 1. Go to the /content/ directory
%cd /content/

# 2. Create my_new_folder in /content/ (if it doesn't exist)
#    You can check first: !ls /content/
# !mkdir my_new_folder

# 3. Navigate into the newly created folder
%cd /content/my_new_folder

# 4. You are now in the folder.
#    You can verify with: %pwd

/content
/content/my_new_folder


In [ ]:
# Assuming you are in the directory where the files you want to move are located
# and my_new_folder is also in this directory.
!mv * my_new_folder/

mv: cannot stat '*': No such file or directory


In [ ]:
%pwd

'/content/my_new_folder'

In [ ]:
!find . -maxdepth 1 -mindepth 1 -not -name 'my_new_folder' -exec mv {} my_new_folder/ \;

In [ ]:
!ls

In [ ]:
!ls /content/my_new_folder

In [ ]:
%cd /content/
!ls -a

/content
 .
 ..
'611005089-pilates-reformer-1 (1).pdf'
 611005089-pilates-reformer-1.pdf
'686451944-NETA-Intro-to-Teaching-Reformer-Pilates-2008 (1).pdf'
 686451944-NETA-Intro-to-Teaching-Reformer-Pilates-2008.pdf
 checkpoints
 combined_data.json
 .config
 drive
 finetuned_arctic_ft
 .git
 .gitignore
 my_new_folder
 pilates_project
 ragas_eval_results.csv
 ragas_golden_eval.csv
 ragas_post_finetune.csv
 sample_data
 test_dataset.json
 training_dataset.json
 val_dataset.json
 wandb


In [ ]:
%cd my_new_folder
!ls -a

/content/my_new_folder
.  ..


In [ ]:
!zip -r /content.zip my_folder_to_zip

	zip warning: name not matched: my_folder_to_zip

zip error: Nothing to do! (try: zip -r /content.zip . -i my_folder_to_zip)


In [ ]:
!ls -I my_new_folder

In [ ]:
cd my_new_folder

[Errno 2] No such file or directory: 'my_new_folder'
/content/my_new_folder


In [ ]:
!git add .